In [1]:
import glob as glob
import pandas as pd
import os
import shutil
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import sem
# from statsmodels.stats.anova import AnovaRM
# import scikit_posthocs as sp
# import statsmodels.formula.api as smf

In [2]:
from scipy import stats

In [3]:
import statsmodels.api as sm

In [4]:
sns.__version__

'0.12.2'

### load started frame info

In [2]:
disk_path="put the disk directory here"

In [3]:
parent_folder=disk_path + "\\CPT_TOT_DS_Stage_3_2nd\\VFA"

In [4]:
frame_cal=pd.read_excel(parent_folder+"\\started frame_2.xlsx")
# frame_cal['vfa_session']=frame_cal['session name'].str[1:] + ".csv"
# frame_cal['group_id']=frame_cal['group_id'].str[1:]

### copy vfa raw file into folder with changed file name

In [5]:
# name_code=pd.read_csv(parent_folder + "\\video and DLC_result name changed\\file_list.csv")
# vfa_raw=name_code['vfa_id'].to_list()
# vfa_recover=name_code['vfa_session'].to_list()

# for i, j in zip (vfa_raw, vfa_recover):
#     shutil.copy2(parent_folder + "\\dlc_output\\results\\%s"%i, parent_folder + "\\dlc_output\results\threshold=3\%s"%j)

### add frames_cal column and other features into each vfa results file 

In [7]:
saveto=parent_folder+"\\dlc_output\\results\\threshold=3\\temp"

if os.path.isdir(saveto) == False:
    os.mkdir(saveto)

os.chdir(parent_folder+"\\dlc_output\\results\\threshold=3")

for i, j, in zip(frame_cal['vfa_session'], frame_cal["start_frame"]):
    
    df=pd.read_csv(i)
    df['frames_cal'] = df['index']-j+1 
    df['sec_cal']=(df["frames_cal"]-1)*1/60+0.001
    
    df["total_visual"]=df["lateralRightright"] + df["lateralLeftright"] + df["frontalright"]
    df['blind_or_not']=np.where(df['blindright']>=0.9, 'blind', 'oriented')
    df=df.round(3)
    
    df.to_csv(saveto+ "\\%s"%(i), index=False)

### change name of abet raw file and add feature

In [1]:
#editing abet ii raw data for syncing:change file name and add feature in file
# origin=parent_folder+ "\\abet raw betch" 

# target=origin + "\\edit_raw_file"

# files=[fileName for fileName in os.listdir(origin) if fileName.endswith(".csv")]

# os.chdir(origin)
# for name in files:
#     if os.path.isdir(target) == False:
#         os.mkdir(target)

#     df=pd.read_csv(name)
#     #df.loc[:, "Schedule run date short"]=pd.to_datetime(df["Date/Time"]).dt.date
#     df["Schedule run date short"]=pd.to_datetime(df["Date/Time"], format="%m/%d/%Y %I:%M:%S %p")
#     df["Schedule run date short"]=df["Schedule run date short"].apply(lambda x: datetime.strftime(x, '%m-%d-%Y'))
#     df["file_name"]=df["Schedule run date short"] + "-" + df["Group ID"] + "-" + df["Animal ID"]
    
#     df=df.drop(['Machine Name', 'Date/Time', 'Version', 'Version Name', 'Application_Version', 
#                 'Experiment', 'Max_Number_Trials', 'Schedule_Description', 'Environment', 
#                 'Analysis Name', 'Schedule Run ID', 'Start ITI - Start'], axis=1)
#     #drop "Start ITI - Start" column, since it impact the orientation determination in next step
#     df=df.melt(id_vars=['Database','Schedule Name', 'Schedule_Start_Time', 'Schedule run date short', 'Animal ID', 'Group ID', 'file_name', 'Max_Schedule_Time'], var_name="event", value_name="sec_cal")
        
#     n=df["file_name"][1]
#     df.to_csv(target + "\\%s.csv"%n)
#     print(df["file_name"][1] + ":Files's names are updated")

### sync abet raw and vfa results

In [9]:
raw_list_=glob.glob(disk_path+"\\CPT_TOT_DS_Stage_3_2nd\\VFA\\abet raw betch\\edit_raw_file\\*.csv")
raw_list_.sort()

target=disk_path+"\\CPT_TOT_DS_Stage_3_2nd\\VFA\\dlc_output\\results\\threshold=3\\temp"
os.chdir(target)
vfa_list=glob.glob(r"*.csv")
vfa_list.sort()

print(len(raw_list_))
print(len(vfa_list))

83
83


In [10]:
raw_path=disk_path+"\\CPT_TOT_DS_Stage_3_2nd\\VFA\\abet raw betch\\edit_raw_file\\" 
raw_list_short_=[fileName for fileName in os.listdir(raw_path) if fileName.endswith(".csv")]
raw_list_short=[i[:16] for i in raw_list_short_]
vfa_list_short=[i[:16] for i in vfa_list]
different = []
for element in raw_list_short:
    if element not in vfa_list_short:
        different.append(element)

different

[]

In [11]:
#check file name match
raw_list=glob.glob(disk_path+"\\CPT_TOT_DS_Stage_3_2nd\\VFA\\abet raw betch\\edit_raw_file\\*.csv")
raw_list.sort()

path=disk_path+"\\CPT_TOT_DS_Stage_3_2nd\\VFA\\dlc_output\\results\\threshold=3\\temp"

os.chdir(path)
vfa_list=glob.glob(r"*.csv")
vfa_list.sort()

d={'raw_list':raw_list, 'vfa_list': vfa_list}
file_list=pd.DataFrame(data=d)

dosage_cond=[(file_list["vfa_list"].str.contains("AMPH-vehicle")),
        (file_list["vfa_list"].str.contains("AMPH-0.3mpk")),
            (file_list["vfa_list"].str.contains("AMPH-0.6mpk")),
            (file_list["vfa_list"].str.contains("AMPH-1mpk")),]
dosage_list=['AMPH-vehicle','AMPH-0.3mpk', 'AMPH-0.6mpk', 'AMPH-1mpk']  
file_list['dosage']=np.select(dosage_cond, dosage_list)
file_list["dt_id"]=file_list["vfa_list"]
for h in range(len(file_list)):
    file_list.loc[:, "dt_id"][h]=file_list.loc[:,'dt_id'][h].replace("-" + file_list.loc[:,'dosage'][h], "")
    

file_list['match']=file_list.apply(lambda x : x.dt_id[:-4] in x.raw_list, axis=1)
file_list['match'].unique()

array([ True])

In [12]:
file_list

,raw_list,vfa_list,dosage,dt_id,match
0,C:\Users\ye.li\OneDrive - Lieber Institute for...,03-29-2022-A-L-2-AMPH-1mpk.csv,AMPH-1mpk,03-29-2022-A-L-2.csv,True
1,C:\Users\ye.li\OneDrive - Lieber Institute for...,03-29-2022-A-R-1-AMPH-vehicle.csv,AMPH-vehicle,03-29-2022-A-R-1.csv,True
2,C:\Users\ye.li\OneDrive - Lieber Institute for...,03-29-2022-B-L-7-AMPH-vehicle.csv,AMPH-vehicle,03-29-2022-B-L-7.csv,True
3,C:\Users\ye.li\OneDrive - Lieber Institute for...,03-29-2022-B-R-8-AMPH-1mpk.csv,AMPH-1mpk,03-29-2022-B-R-8.csv,True
4,C:\Users\ye.li\OneDrive - Lieber Institute for...,03-29-2022-B-RL-6-AMPH-0.6mpk.csv,AMPH-0.6mpk,03-29-2022-B-RL-6.csv,True
...,...,...,...,...,...
78,C:\Users\ye.li\OneDrive - Lieber Institute for...,04-14-2022-Z-R-16-AMPH-0.6mpk.csv,AMPH-0.6mpk,04-14-2022-Z-R-16.csv,True
79,C:\Users\ye.li\OneDrive - Lieber Institute for...,04-19-2022-W-L-2-AMPH-vehicle.csv,AMPH-vehicle,04-19-2022-W-L-2.csv,True
80,C:\Users\ye.li\OneDrive - Lieber Institute for...,04-19-2022-Z-R-16-AMPH-0.3mpk.csv,AMPH-0.3mpk,04-19-2022-Z-R-16.csv,True
81,C:\Users\ye.li\OneDrive - Lieber Institute for...,04-21-2022-W-L-2-AMPH-1mpk.csv,AMPH-1mpk,04-21-2022-W-L-2.csv,True


### sync abet raw data to vfa result

In [14]:
# merge cpt raw data and vfa data to sync event
#save processed file into the pathway below:

pathfile=disk_path+"\\CPT_TOT_DS_Stage_3_2nd\\VFA\\dlc_output\\results\\threshold=3\\temp_2_V2"
if os.path.isdir(pathfile) == False:
    os.mkdir(pathfile)

target=disk_path+"\\CPT_TOT_DS_Stage_3_2nd\\VFA\\dlc_output\\results\\threshold=3\\temp"
os.chdir(target)

for i, j in zip(raw_list, vfa_list):
    df=pd.read_csv(i, index_col=None, header=0)
    evnt=df[["event", "sec_cal"]].copy()
    evnt.dropna(subset=['sec_cal'], inplace=True)
    evnt["count"]=1
    #load vfa data and reform the format of the dataframe    
    df=pd.read_csv(j)

    #seperating handling Center Screen Touch - Time event, since it comes with Hit, 
    #and makes the orientated hit tobe undeterminded
   
    #merge cpt raw data, not includes center screen touch event and Stim Onset - End ITI, and vfa data
    combine_1=pd.concat([evnt[(evnt["event"]!="Center Screen Touch - Time") & (evnt["event"]!="Stim Onset - End ITI")], df], 
                        ignore_index=False).sort_values(by='sec_cal').reset_index()

    #check each row, add ori_before and ori_after
    combine_1['ori_before']=""
    combine_1["ori_after"]=""

    for h in range(len(combine_1)):# the code in combine_1, combine_2, combine_3 are updated to fix the chained index issue
        if (combine_1.loc[h, "count"]==1 and h<len(combine_1)-1):
            combine_1.loc[h, "ori_before"]=combine_1.loc[h-1,'blind_or_not']
            combine_1.loc[h, "ori_after"]=combine_1.loc[h+1, 'blind_or_not']
            combine_1['ori'] = np.where(combine_1["ori_before"] == combine_1["ori_after"], combine_1['ori_after'], "undetermined")

        elif (combine_1.loc[h, "count"]==1 and h==len(combine_1)-1):
            print("combine_1/evnet is the last frame: "+j)   
            combine_1.loc[h, "ori_before"]=combine_1.loc[h-1,'blind_or_not']
            combine_1.loc[h, 'ori'] = combine_1.loc[h, "ori_before"]

    #merge cpt raw data only includes center screen touch event and vfa data
    combine_2=pd.concat([evnt[evnt["event"]=="Center Screen Touch - Time"], df], ignore_index=False).sort_values(by='sec_cal').reset_index()

    #check each row, add ori_before and ori_after
    combine_2['ori_before']=""
    combine_2["ori_after"]=""

    for h in range(len(combine_2)):
        if (combine_2.loc[h, "count"]==1 and h<len(combine_2)-1):
            combine_2.loc[h, "ori_before"]=combine_2.loc[h-1,'blind_or_not']
            combine_2.loc[h, "ori_after"]=combine_2.loc[h+1, 'blind_or_not']
            combine_2['ori'] = np.where(combine_2["ori_before"] == combine_2["ori_after"], combine_2['ori_after'], "undetermined")

        elif (combine_2.loc[h, "count"]==1 and h==len(combine_2)-1):
            print("combine_1/evnet is the last frame: "+j) 
            combine_2.loc[h, "ori_before"]=combine_2.loc[h-1,'blind_or_not']
            combine_2.loc[h, 'ori'] = combine_2.loc[h, "ori_before"]

    #merge cpt raw data only Stim Onset - End ITI and vfa data
    combine_3=pd.concat([evnt[evnt["event"]=="Stim Onset - End ITI"], df], ignore_index=False).sort_values(by='sec_cal').reset_index()

    #check each row, add ori_before and ori_after
    combine_3['ori_before']=""
    combine_3["ori_after"]=""

    for h in range(len(combine_3)):
        if (combine_3.loc[h, "count"]==1 and h<len(combine_3)-1):
            combine_3.loc[h, "ori_before"]=combine_3.loc[h-1,'blind_or_not']
            combine_3.loc[h, "ori_after"]=combine_3.loc[h+1, 'blind_or_not']
            combine_3['ori'] = np.where(combine_3["ori_before"] == combine_3["ori_after"], combine_3['ori_after'], "undetermined")   

        elif (combine_3.loc[h, "count"]==1 and h==len(combine_3)-1):
            print("combine_1/evnet is the last frame: "+j) 
            combine_3.loc[h, "ori_before"]=combine_3.loc[h-1,'blind_or_not']
            combine_3.loc[h, 'ori'] = combine_3.loc[h, "ori_before"]

    #combine the two part results
    combine=pd.concat([combine_1, combine_2[combine_2["event"]=="Center Screen Touch - Time"], 
                       combine_3[combine_3["event"]=="Stim Onset - End ITI"]], ignore_index=False).sort_values(by='sec_cal')

    combine=combine.drop(['level_0'], axis=1)
    
    #add 45min bin and 15min bin feature

    conditions_15=[(combine["sec_cal"]<0.000),
            (combine["sec_cal"]>=0.000) & (combine["sec_cal"]<=900.000),
            (combine["sec_cal"]>900.000) & (combine["sec_cal"]<=1800.000), 
            (combine["sec_cal"]>1800.000) & (combine["sec_cal"]<=2700.000),
            (combine["sec_cal"]>2700.000) & (combine["sec_cal"]<=3600.000),
            (combine["sec_cal"]>3600.000) & (combine["sec_cal"]<=4500.000),
            (combine["sec_cal"]>4500.000) & (combine["sec_cal"]<=5400.000),
                  (combine["sec_cal"]>5400.000)]

    values_15=['before_session','15bin_1', '15bin_2', '15bin_3', '15bin_4', '15bin_5', '15bin_6', 'after_session'] 

    conditions_45=[(combine["sec_cal"]<0.000),
                   (combine["sec_cal"]>=0.000) & (combine["sec_cal"]<=2700.000),
            (combine["sec_cal"]>2700.000) & (combine["sec_cal"]<=5400.000), (combine["sec_cal"]>5400.000)]

    values_45=['before_session',"first_45min", "last_45min", "after_session"] 
    
       
    combine['45min_bin']=np.select(conditions_45, values_45)
    
    combine['15min_bin']=np.select(conditions_15, values_15)
    
    combine=combine.round(3)
    combine['cohort']='coh2'
    
    combine.to_csv(pathfile + "\\%s"%j)

combine_1/evnet is the last frame: 04-05-2022-Z-L-15-AMPH-1mpk.csv
combine_1/evnet is the last frame: 04-05-2022-Z-L-15-AMPH-1mpk.csv
combine_1/evnet is the last frame: 04-05-2022-Z-L-15-AMPH-1mpk.csv
